In [ ]:
!pwd

import paths_2D as paths
import loss_2D as loss
import model_ronneberger_512 as model
import glob
import numpy as np
import random
from matplotlib import pyplot as plt

In [ ]:
def plot_batch_predict(x=20, model=model, test_imgs=test_imgs, test_masks=test_masks):
    y_input = test_imgs[x:x+5]
    y_true = test_masks[x:x+5]

    print("Input shape:", y_input.shape)
    print("Truth shape: ", y_true.shape)

    y_pred = model.predict(y_input)
    print("Predict shape:", y_pred.shape)

    y_pred_rounded = np.round(y_pred)
    diff = y_pred_rounded - y_true

    CMAP = 'binary'
    
    for batch_index in range(y_input.shape[0]):
        fig, axs = plt.subplots(1, 5, figsize=(15,3), sharey=True)
        
        for ax in axs:
            ax.set_xticks([])
            ax.set_yticks([])

        axs[0].imshow(np.log(1 + y_input[batch_index,...,0]), cmap=CMAP)
        axs[0].set_title("Input")
        axs[0].contour(y_true[batch_index,...,0])
    
        axs[1].imshow(y_true[batch_index,...,0], cmap=CMAP)
        axs[1].set_title("Truth")
        
        axs[2].imshow(y_pred[batch_index,...,0], cmap=CMAP)
        axs[2].set_title("Probabilistic map")

        axs[3].imshow(y_pred_rounded[batch_index,...,0], cmap=CMAP)
        axs[3].set_title("Binary prediction")
        
        axs[4].imshow(diff[batch_index,...,0], cmap=CMAP)
        axs[4].set_title("Diff: pred-truth")

In [ ]:
#path = "/home/matthew/datasets/vet_dataset_cleaned_npy/"
path = "/home/jupyter/vet_dataset_cleaned_npy/"
img_paths = glob.glob(path + "img/*")
mask_paths = glob.glob(path + "mask/*")

img_paths.sort()
mask_paths.sort()

temp = list(zip(img_paths, mask_paths))

random.shuffle(temp)

img_paths, mask_paths = zip(*temp)

print(img_paths[0])
print(mask_paths[0])

In [ ]:
# SMALLER SET
# img_paths = img_paths[0:200]
# mask_paths = mask_paths[0:200]

In [ ]:
imgs = np.array([np.load(x) for x in img_paths])

imgs_mean = np.mean(imgs)
imgs_std = np.std(imgs)
imgs = (imgs - imgs_mean) / imgs_std
imgs = imgs[...,np.newaxis]

swap = len(imgs)-100
train_imgs = imgs[0:swap]
test_imgs = imgs[swap:-1]

In [ ]:
masks = [np.load(x) for x in mask_paths]

for index, mask in enumerate(masks):
    if masks[index].shape == (512, 512, 0):
        masks[index] = np.zeros((512, 512, 1))

masks = np.array(masks)
train_masks = masks[0:swap]
test_masks = masks[swap:-1]


In [ ]:
print(imgs.shape)
print(masks.shape)
print("--------")
print(train_imgs.shape)
print(train_masks.shape)
print("--------")
print(test_imgs.shape)
print(test_masks.shape)
print("--------")
print(train_imgs[0].shape)
print(train_masks[0].shape)

In [ ]:
import tensorflow as tf
import model_ronneberger_512 as model

MODEL_SAVE = "./weights/bce_512.{epoch:02d}.hdf5"

LOSS = tf.keras.losses.BinaryCrossentropy()
INITIAL_LR = 1e-4
STOPPING_PATIENCE = 50
LR_PATIENCE = 4

METRICS = [loss.dsc_loss]
OPTIMIZER = tf.keras.optimizers.Adam(lr = INITIAL_LR)
LR_SCALE = 0.5


early_stopping = tf.keras.callbacks.EarlyStopping(patience=STOPPING_PATIENCE, verbose=1, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=LR_SCALE, patience=LR_PATIENCE, verbose=1)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(MODEL_SAVE, save_weights_only=True, verbose=1)

CALLBACKS = [early_stopping, model_checkpoint, reduce_lr]

model = model.model(train_imgs[0].shape[0], train_masks[0].shape[-1])

model.compile(optimizer = OPTIMIZER, loss = LOSS ,metrics = METRICS)

In [ ]:
# model.summary()

In [ ]:
from matplotlib import pyplot as plt
x = 20

y_input = train_imgs[x:x+5]
y_true = train_masks[x:x+5]

print("Input shape:", y_input.shape)
print("Truth shape: ", y_true.shape)


y_pred = model.predict(y_input)
#y_pred = np.round(y_pred)
print("Predict shape:", y_pred.shape)


for batch_index in range(y_input.shape[0]):
    fig, axs = plt.subplots(1, 4, figsize=(20,80))
    axs[0].imshow(y_input[batch_index,...,0])
    
    axs[1].imshow(y_true[batch_index,...,0])
    axs[2].imshow(y_pred[batch_index,...,0])

    axs[3].imshow(y_true[batch_index,...,0], alpha=0.5)
    axs[3].imshow(y_pred[batch_index,...,0], alpha=0.5)

In [ ]:
EPOCHS = 1

print("\n Training...")
train_history = model.fit(x=train_imgs, 
                          y=train_masks,
                          batch_size=5,
                          epochs=EPOCHS, 
                          callbacks=CALLBACKS, 
                          validation_split=0.2)

In [ ]:
plot_batch_predict()

In [ ]:
import tensorflow as tf
import model_ronneberger_512 as model

MODEL_SAVE = "./weights_vacbag/dice_after_1_bce_512.{epoch:02d}.hdf5"

LOSS = loss.dsc_loss
INITIAL_LR = 1e-5
STOPPING_PATIENCE = 50
LR_PATIENCE = 4

AUC = tf.keras.metrics.AUC(
    num_thresholds=200, curve='ROC', summation_method='interpolation', name=None,
    dtype=None, thresholds=None, multi_label=False, label_weights=None
)

METRICS = [loss.dice_metric, tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), AUC]
OPTIMIZER = tf.keras.optimizers.Adam(lr = INITIAL_LR)
LR_SCALE = 0.5


early_stopping = tf.keras.callbacks.EarlyStopping(patience=STOPPING_PATIENCE, verbose=1, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=LR_SCALE, patience=LR_PATIENCE, verbose=1)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(MODEL_SAVE, save_weights_only=True, verbose=1)

CALLBACKS = [early_stopping, model_checkpoint, reduce_lr]

model = model.model(train_imgs[0].shape[0], train_masks[0].shape[-1])

model.compile(optimizer = OPTIMIZER, loss = LOSS ,metrics = METRICS)

model.load_weights("./weights/bce_512.01.hdf5")

In [ ]:
EPOCHS = 100

print("\n Training...")
train_history = model.fit(x=train_imgs, 
                          y=train_masks,
                          batch_size=5,
                          epochs=EPOCHS, 
                          callbacks=CALLBACKS, 
                          validation_split=0.2)

In [ ]:
#Epoch 00011: saving model to ./weights/dice_after_1_bce_512.11.hdf5

plot_batch_predict()

In [ ]:
import tensorflow as tf
import model_ronneberger_512 as model

MODEL_SAVE = "./weights_vacbag/dice_2_after_1_bce_512.{epoch:02d}.hdf5"

LOSS = loss.dsc_loss
INITIAL_LR = 1e-6
STOPPING_PATIENCE = 50
LR_PATIENCE = 4

METRICS = [loss.dice_metric,
           tf.keras.metrics.Precision(),
           tf.keras.metrics.Recall()]

OPTIMIZER = tf.keras.optimizers.Adam(lr = INITIAL_LR)
LR_SCALE = 0.5


early_stopping = tf.keras.callbacks.EarlyStopping(patience=STOPPING_PATIENCE, verbose=1, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=LR_SCALE, patience=LR_PATIENCE, verbose=1)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(MODEL_SAVE, save_weights_only=True, verbose=1)

CALLBACKS = [early_stopping, model_checkpoint, reduce_lr]

model = model.model(train_imgs[0].shape[0], train_masks[0].shape[-1])

model.compile(optimizer = OPTIMIZER, loss = LOSS ,metrics = METRICS)

model.load_weights("./weights/dice_after_1_bce_512.11.hdf5")

EPOCHS = 100

print("\n Training...")
train_history = model.fit(x=train_imgs, 
                          y=train_masks,
                          batch_size=5,
                          epochs=EPOCHS, 
                          callbacks=CALLBACKS, 
                          validation_split=0.2)

In [ ]:
plot_batch_predict(x=80)

In [ ]:
model.evaluate(x=test_imgs,
              y=test_masks,
              batch_size=5)